In [2]:
import pandas as pd
import re
import numpy as np
import requests
import glob
import os
from tqdm import tqdm
from janitor import clean_names
from dicts import dict_columns_rrhh_download, dict_columns_select_download, path
from openpyxl import load_workbook
from datetime import datetime
import locale as lc

In [4]:
def download_excel(files, path_download):
    """
        Extrae código, razon_social, rut y declaración de retención de impuestos y pagos provicionales codigos: 538, 142 y 020

        Args:
            files (str): path al archivo a extraer
    """
    
    try: 

        sheet_patron = r"BD_Postulación|BD_postulacion|BD postulacion|BD postulación"

        nombre_hoja = next(sheet for sheet in pd.ExcelFile(files).sheet_names if re.search(sheet_patron, sheet, flags=re.IGNORECASE))

        df = pd.read_excel(files,
                    sheet_name=nombre_hoja).clean_names()
        try:
            try:

                patron = "company_name|application_name"  # Patrón para identificar la fila

                fila_nombres = df[df.apply(lambda row: row.str.contains(patron, case=False, na=False)).any(axis=1)].index[0]

            except:
                patron = "application_name"  # Patrón para identificar la fila

                fila_nombres = df[df.eq(patron).any(axis=1)].index[0]
            
            df = pd.read_excel(files,
                    sheet_name=nombre_hoja,
                    header=fila_nombres+1).clean_names()

            inverted_dict_column = {value: key for key, values in dict_columns_rrhh_download.items() for value in values}
                
            df = df.rename(columns=inverted_dict_column)

            df = df.filter(items=dict_columns_select_download)

        except:
            inverted_dict_column = {value: key for key, values in dict_columns_rrhh_download.items() for value in values}
            
            df = df.rename(columns=inverted_dict_column)

            df = df.filter(items=dict_columns_select_download)
    
    except:

        # sheet_patron = r"BD_Postulación|BD_postulacion|BD postulacion|BD postulación"

        # nombre_hoja = next(sheet for sheet in pd.ExcelFile(files).sheet_names if re.search(sheet_patron, sheet, flags=re.IGNORECASE))

        # df = pd.read_excel(files, sheet_name=sheet_patron).clean_names()

        df = pd.read_excel(files).clean_names()

        inverted_dict_column = {value: key for key, values in dict_columns_rrhh_download.items() for value in values}
            
        df = df.rename(columns=inverted_dict_column)

        #df = df[(df.application_sent == 'si') & (~df.plan_trabajo_presupuesto_rrhh.isna())]

        try:
            df = df.filter(items=dict_columns_select_download)

        except:
            df = df.loc[:, dict_columns_select_download]

    for index, row in df.iterrows():
        codigo = str(row['codigo'])
        carpeta_destino = f'{path_download}/{codigo}'

        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)

        hipervinculo = row['plan_trabajo_presupuesto_rrhh']
    
        nombre_archivo = codigo + '.xlsx'
        
        ruta_archivo = os.path.join(carpeta_destino, nombre_archivo)
        
        if os.path.exists(ruta_archivo):
            continue

        try:
            response = requests.get(hipervinculo)
            with open(ruta_archivo, 'wb') as file:
                file.write(response.content)
        except Exception as e:
            print(f"Error al descargar el archivo desde {hipervinculo}: {str(e)}")
            continue

In [5]:
path = r'data\2.2023\CV'

In [6]:
def rrhh_scrap(path, año=2023):
    """
        Recorre carpetas por año para extraer RRHH reportados por beneficiarios en base al código del proyecto

        Args:
            path (str): path a los archivos que contienen las BD de postulación con los archivos de RRHH postulados.
            año (int): año de la convocatoria
    """
    
    for concurso in tqdm(glob.iglob(f'{path}/*')):
        print(concurso)
        download_excel(concurso, path_download = 'output/2.2023/CV')

In [9]:
rrhh_scrap(path)

0it [00:00, ?it/s]

data\2.2023\CV\2.2023 CV Base de datos de postulación.xlsx


1it [00:02,  2.85s/it]


# Cambios nombres de hoja de presupuesto

In [ ]:
path = r'data/2.2023/CV/2.2023 CV Base de datos de postulación.xlsx'

In [ ]:
df = pd.read_excel(path).clean_names()

In [ ]:
df.columns.to_list()

## **Proceso de Inserción de datos asociados a PPTO instrumento "CREA Y VALIDA"**

A continuación, se implementará un sistema que nos permite obtener información relevante del presupuesto de los proyectos.

Entre los hitos fundamentales que se realizarán son:
- El código del proyecto obtenido de Charly y SGP.
- Aporte Innova Chile (Subsidio) $
- Aporte Beneficiaria (Pecuniario) $
- Aporte Beneficiaria (Valorado) $ 
- Aporte Asociados (Pecuniario) $ 
- Aporte Asociados (Valorado) $ 
- Total ($). Corresponde al monto total del proyecto
- Aporte InnovaChile Etapa Crea
- Aporte InnovaChile Etapa Valida
- Gasto de Inversión
- Gastos de Administración


### **Pasos de ejecución**

#### **1-. Mostrar la ruta absoluta según perfil**
 - Ejecutar celda para mostrar la ruta principal según perfil de usuario.
 - El resultado mostrará la ruta  del usuario registrado.

In [3]:
# Se obtiene la ruta principal, que es llamada del directorio. Acá se pueden crear n rutas para distintos perfiles.
path['esteban']

'C:/Users/esteban.berrios/OneDrive - corfo.cl/'

#### **2-. Guardar la ruta en donde se encuentran guardados los proyectos**
- Ejecutar celda para llamar la ruta absoluta asociada a mi perfil, y concatenarla con la ruta relativa asociada al directorio donde están los proyectos.
- En la variable input_path quedará guardada la ruta de acceso a los proyectos.

In [4]:
# Tabla con presupuestos. El códigop permite generar nueva ruta llamada input_path que apunta a un directorio específico
# dentro de la estructura de directorios en path['esteban']. Esto facilita la manipulación y navegación en el sistema de archivos dentro de ese directorio en particular.
ruta = os.path.abspath(path['esteban'])
input_path = os.path.join(ruta, "extract_pdt/output/2.2023/CV")

#### **3-. Mostrar la nueva ruta concatenada correspondiente al directorio en donde se guardan los proyectos**
- La ejecución de la celda mostrará la ruta asociada al directorio en donde se guardan los proyectos en formato .xlsx.

In [5]:
input_path #Muestra la nueva ruta.

'C:\\Users\\esteban.berrios\\OneDrive - corfo.cl\\extract_pdt/output/2.2023/CV'

#### **4-. Mostrar el acceso a la carpeta asociada a los proyectos**
- El resultado de la ejecución de esta celda, permite mostrar el llamado del primer proyecto que se encuentra en la carpeta.

In [6]:
#  Se define la variable folder como un vacío, y  con la siguiente línea de toma el primer archivo encontrado en una carpeta específica (dada por input_path y folder), 
# elimina la extensión ".xlsx" del nombre de ese archivo y devuelve el resultado, que probablemente sea utilizado como un identificador o código para ese archivo en particular.
folder = ""
os.listdir(os.path.join(input_path,folder))[0].replace(".xlsx","")

'23CV2-251359'

#### **5-.Guardar en un Primer Data Frame la información correspondiente a los indicadores de presupuesto de los proyectos**
A partir de la ejcución de esta celda que contiene la estructura de código podrá obtener un Data Frame que tendrá de todos los proyectos:
- El código obtenido de Charly y SGP.
- Aporte Innova Chile (Subsidio) $
- Aporte Beneficiaria (Pecuniario) $
- Aporte Beneficiaria (Valorado) $ 
- Aporte Asociados (Pecuniario) $ 
- Aporte Asociados (Valorado) $ 
- Total ($). Corresponde al monto total del proyecto
- Aporte InnovaChile Etapa Crea
- Aporte InnovaChile Etapa Valida
- Gasto de Inversión
- Gastos de Administración

El resultado se guardará en el data frame llamado df_presupuesto.

In [7]:
%%capture
%%time
df_presupuesto = pd.DataFrame()

for folder in os.listdir(input_path):
    folder_path = os.path.join(input_path, folder)
    if os.path.isdir(folder_path):
        excel_files = [file for file in os.listdir(folder_path) if file.endswith(".xlsx")]

        for excel_file in excel_files:
            file = os.path.join(folder_path, excel_file)
            codigo = os.listdir(os.path.join(input_path, folder))[0].replace(".xlsx", "")
            
            try:
                # Lectura del archivo Excel
                worksheet = pd.read_excel(file, sheet_name="RESUMEN PPTO")

                # Resto del código para procesar los datos cuando no hay excepción
                worksheet = worksheet.iloc[7:13, 1:]
                worksheet.columns = worksheet.iloc[0]
                worksheet = worksheet[1:].reset_index(drop=True)
                # String que captura nombre de variable Cuenta (Financiable o Presupuestable)
                str_cuenta_financiable = worksheet.columns[worksheet.columns.str.contains("Cuenta")].values[0]
                worksheet_row = worksheet[worksheet[str_cuenta_financiable] == "TOTAL ($)"]
                fila_gasto_inversion = worksheet[worksheet[str_cuenta_financiable] == "Gasto de Inversión"]
                indice_gasto_inversion = fila_gasto_inversion.index[0]  # Obtenemos el índice de la fila "Gasto de Inversión"
                registro_gasto_inversion = fila_gasto_inversion.iloc[0,1]
                fila_gasto_administracion = worksheet[worksheet[str_cuenta_financiable] == "Gastos de Administración"]
                indice_gasto_administracion = fila_gasto_administracion.index[0]  # Obtenemos el índice de la fila "Gasto de Inversión"
                registro_gasto_administracion = fila_gasto_administracion.iloc[0,1]

                # Anexar los datos al DataFrame df_presupuesto
                worksheet_row.insert(0, "codigo", codigo) # Agregar la columna 'codigo' en la primera ubicación a la izquierda
                worksheet_row.insert(worksheet_row.columns.__len__(),"Gasto de Inversión",registro_gasto_inversion)
                worksheet_row.insert(worksheet_row.columns.__len__(),"Gastos de Administración",registro_gasto_administracion)
                worksheet_row.drop(columns=[str_cuenta_financiable], inplace=True) # Eliminar la columna 'Cuentas Financiables'
                df_presupuesto = pd.concat([df_presupuesto, worksheet_row])

                # Agregar la columna "Estado" y registrar "ok" para esta fila
                df_presupuesto.loc[df_presupuesto['codigo'] == codigo, 'Estado'] = 'ok'

            except Exception as e:
                # Si se produce una excepción al leer el archivo Excel, se imprime un mensaje de error
                print(f'Error al procesar el archivo {file}: {str(e)}')

                # Crear un diccionario con el código y valores 0 para todas las columnas, excepto 'codigo'
                data = {'codigo': codigo, 'Estado': 'inadmisible'}
                for col in df_presupuesto.columns:
                    if col != 'codigo' and col != 'Estado':
                        data[col] = 0

                # Agregar una nueva fila al DataFrame df_presupuesto
                df_presupuesto = df_presupuesto.append(data, ignore_index=True)
                continue  # Continúa con el siguiente archivo

# EN CASO DE QUE REQUIERA QUE LOS VALORES SE LE DE UN FORMATO MONETARIO, QUITAR EL COMENTARIO ASOCIADO.
# lc.setlocale(lc.LC_ALL, 'es_CL.UTF-8')

# def format_currency(value):
#     return lc.currency(value, grouping=True, symbol='$')

# # Aplicar formato de moneda a las columnas numéricas (excepto 'codigo' y 'Estado')
# columnas_numericas = df_presupuesto.columns.difference(['codigo', 'Estado'])
# df_presupuesto[columnas_numericas] = df_presupuesto[columnas_numericas].applymap(format_currency)

# Reiniciar el índice del DataFrame resultante
df_presupuesto.reset_index(drop=True, inplace=True)

#### **6-.Mostrar el resultado del Data Frame con la información correspondiente a los indicadores de presupuesto de los proyectos**
Se realiza el desglose presupuestario de los proyectos revisados.

In [10]:
df_presupuesto.head(20)

,codigo,Aporte Innova Chile\n(Subsidio) $,Aporte Beneficiaria\n(Pecuniario) $,Aporte Beneficiaria\n(Valorado) $,Aporte Asociados\n(Pecuniario) $,Aporte Asociados\n(Valorado) $,Total ($),Aporte Innova Chile Etapa Crea,Aporte Innova Chile Etapa Valida,Gasto de Inversión,Gastos de Administración,Estado
0,23CV2-251359,0,0,0,0,0,0,0,0,0.0,0.0,ok
1,23CV2-251360,0,0,0,0,0,0,0,0,0.0,0.0,ok
2,23CV2-251361,104128000,12996000,29562000,0,0,146686000,48096000,56032000,8800000.0,0.0,ok
3,23CV2-251362,177038670,22599960,22500180,0,0,222138810,70868320,106170350,4700000.0,1800000.0,ok
4,23CV2-251363,0,0,0,0,0,0,0,0,0.0,0.0,inadmisible
5,23CV2-251364,0,0,0,0,0,0,0,0,0.0,0.0,inadmisible
6,23CV2-251365,127599968,32400000,0,0,0,159999968,82899976,44699992,38199984.0,800000.0,ok
7,23CV2-251366,0,0,0,0,0,0,0,0,0.0,0.0,ok
8,23CV2-251367,214263600,28210000,25908800,0,0,268382400,125906800,88356800,35000000.0,0.0,ok
9,23CV2-251368,0,0,0,0,0,0,0,0,0.0,0.0,inadmisible


#### **Crear una nueva hoja de trabajo o worksheet en el documento xlsx que contendrá el resultado final del data frame importado a excel**
A partir de la ejcución de esta celda se podrá obtener:
-La importación del resultado final del Data Frame a un worksheet definido según la veriable del archivo excel y el nombre de la hoja de trabajo que contendrá el resultado.

In [9]:
nombre_archivo_excel = 'data/2.2023/CV/2.2023 CV Base de datos de postulación.xlsx'
nombre_hoja_excel = 'PPTO'  # Cambia 'Hoja1' al nombre de tu hoja específica

# Crea un objeto ExcelWriter para escribir en el archivo Excel
with pd.ExcelWriter(nombre_archivo_excel, engine='openpyxl', mode='a') as writer:
    # Carga el DataFrame en la hoja de Excel especificada
    df_presupuesto.reset_index().to_excel(writer, sheet_name=nombre_hoja_excel, index=False)# Usar EL .reset_index() en caso que desea que el indice aparezca en el df.
    #df_presupuesto.reset_index(drop=True).to_excel(writer, sheet_name=nombre_hoja_excel, index=False)# Al resetear el indice, automáticamente queda asociado como columna. SE DEBE USAR PARA INSTRUMENTOSQUE NO CONTEMPLEN APORTE I+D.

#### **7-. Analizar y guardar en un segundo DataFrame llamado df_gasto_e_c todos los asociados a Entidades Colaboradoras tanto en la etapa Crea y Valida.**
El resultado de este análisis se obtiene:
- Gasto asociado a Entidades Colaboradoras ETAPA CREA
- Gasto asociado a Entidades Colaboradoras ETAPA VALIDA
- Una columna llamada CONDICIÓN, que analiza si existe algún incumplimiento de los proyecto frente a estos requerimieentos.


In [9]:
%%capture
%%time

# Crear un DataFrame para almacenar los datos
df_gasto_e_c = pd.DataFrame()

for folder in os.listdir(input_path):
    folder_path = os.path.join(input_path, folder)
    if os.path.isdir(folder_path):
        excel_files = [file for file in os.listdir(folder_path) if file.endswith(".xlsx")]
        
        for excel_file in excel_files:
            file = os.path.join(folder_path, excel_file)
            codigo = os.listdir(os.path.join(input_path, folder))[0].replace(".xlsx", "")

            try:
                # Lectura del archivo Excel
                worksheet = pd.read_excel(file, sheet_name="OPERACION")

                etapa_crea_rows = worksheet[worksheet.iloc[:, 1] == "ETAPA CREA"]
                etapa_valida_rows = worksheet[worksheet.iloc[:, 1] == "ETAPA VALIDA"]

                if not etapa_crea_rows.empty and not etapa_valida_rows.empty:
                    start_row_crea = etapa_crea_rows.index[0]
                    start_row_valida = etapa_valida_rows.index[0]

                    valor_crea = worksheet.iloc[[start_row_crea + 2, start_row_crea + 3], :].reset_index(drop=True)
                    valor_crea.columns = valor_crea.iloc[0]
                    valor_crea = valor_crea[1:]

                    valor_valida = worksheet.iloc[[start_row_valida + 2, start_row_valida + 3], :].reset_index(drop=True)
                    valor_valida.columns = valor_valida.iloc[0]
                    valor_valida = valor_valida[1:]

                    # Crear un DataFrame temporal para los datos de este archivo
                    temp_df = pd.DataFrame({'codigo': codigo,
                                            'Gasto asociado a Entidades Colaboradoras ETAPA CREA': valor_crea['TOTAL ($)'],
                                            'Gasto asociado a Entidades Colaboradoras ETAPA VALIDA': valor_valida['TOTAL ($)']})

                    # Concatenar el DataFrame temporal con df_gasto_e_c
                    df_gasto_e_c = pd.concat([df_gasto_e_c, temp_df], ignore_index=True)
                    df_gasto_e_c.loc[df_gasto_e_c['codigo'] == codigo, 'Condición'] = 'ok'

                else:
                    print(f"No se encontraron datos válidos en el archivo: {excel_file}")
                    data_e_c = {'codigo': codigo, 'Condición': 'Revisar'}
                    for col in df_gasto_e_c.columns:
                        if col != 'codigo' and col != 'Condición':
                            data_e_c[col] = 0
                    df_gasto_e_c = df_gasto_e_c.append(data_e_c, ignore_index=True)                 

            except Exception as e:
                # Si se produce una excepción al leer el archivo Excel, imprimir un mensaje de error
                print(f'Error al procesar el archivo {file}: {str(e)}')
                data = {'codigo': codigo, 'Condición': 'Revisar'}
                for col in df_gasto_e_c.columns:
                    if col != 'codigo' and col != 'Condición':
                        data[col] = 0

                # Agregar una nueva fila al DataFrame df_presupuesto
                df_gasto_e_c = df_gasto_e_c.append(data, ignore_index=True)
                continue  # Continúa con el siguiente archivo

# df_gasto_e_c ahora contiene todos los datos procesados
df_gasto_e_c


#### **8-. Concatenar los resultados obtenidos en los DataFrames df_presupuesto y df_gasto_e_c dando el resultado final del análisis solicitado.**
El nuevo Data Frame entrega un resultado final con la información de los presupuestos y los gastos asociados a entidad colaboradora. 



In [10]:
df_resultado = pd.concat([df_presupuesto.set_index('codigo'), df_gasto_e_c.set_index('codigo')], axis=1)

In [11]:
df_resultado

,Aporte Innova Chile\n(Subsidio) $,Aporte Beneficiaria\n(Pecuniario) $,Aporte Beneficiaria\n(Valorado) $,Aporte Asociados\n(Pecuniario) $,Aporte Asociados\n(Valorado) $,Total ($),Aporte Innova Chile Etapa Crea,Aporte Innova Chile Etapa Valida,Gasto de Inversión,Gastos de Administración,Estado,Gasto asociado a Entidades Colaboradoras ETAPA CREA,Gasto asociado a Entidades Colaboradoras ETAPA VALIDA,Condición
codigo,,,,,,,,,,,,,,
23CV2-251359,0,0,0,0,0,0,0,0,0.0,0.0,ok,0,0,ok
23CV2-251360,0,0,0,0,0,0,0,0,0.0,0.0,ok,0,0,ok
23CV2-251361,104128000,12996000,29562000,0,0,146686000,48096000,56032000,8800000.0,0.0,ok,0,0,ok
23CV2-251362,177038670,22599960,22500180,0,0,222138810,70868320,106170350,4700000.0,1800000.0,ok,0,0,ok
23CV2-251363,0,0,0,0,0,0,0,0,0.0,0.0,inadmisible,0,0,Revisar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23CV2-251813,169120000,25500000,25200000,0,0,219820000,119080000,50040000,0.0,0.0,ok,0,0,ok
23CV2-251814,180001823,45601770,0,0,0,225603593,124816773,55185050,2560000.0,0.0,ok,0,0,ok
23CV2-251815,178148000,45340000,0,0,0,223488000,105324000,72824000,0.0,0.0,ok,19500000,0,ok


#### **9-.Crear una nueva hoja de trabajo o worksheet en el documento xlsx que contendrá el resultado final del data frame importado a excel**
A partir de la ejcución de esta celda se podrá obtener:
-La importación del resultado final del Data Frame a un worksheet definido según la veriable del archivo excel y el nombre de la hoja de trabajo que contendrá el resultado.

In [220]:
nombre_archivo_excel = 'data/2.2023/CV/2.2023 CV Base de datos de postulación.xlsx'
nombre_hoja_excel = 'PRESUPUESTO'  # Cambia 'Hoja1' al nombre de tu hoja específica

# Crea un objeto ExcelWriter para escribir en el archivo Excel
with pd.ExcelWriter(nombre_archivo_excel, engine='openpyxl', mode='a') as writer:
    # Carga el DataFrame en la hoja de Excel especificada
    df_resultado.reset_index().to_excel(writer, sheet_name=nombre_hoja_excel, index=False)# Usar EL .reset_index() en caso que desea que el indice aparezca en el df.
    #df_presupuesto.reset_index(drop=True).to_excel(writer, sheet_name=nombre_hoja_excel, index=False)# Al resetear el indice, automáticamente queda asociado como columna. SE DEBE USAR PARA INSTRUMENTOSQUE NO CONTEMPLEN APORTE I+D.

